In [ ]:
# default_exp core

# module name here

> API details.

---

## To-Do and Ideas

#### Data
* new confirmed cases / day
* number of test / day
* % of positive tests / day
* new icu / day
* new resp / day
* new death / day
* new and total quarantined

#### Features
* completness of data (what about missing days?)
* hotspots: automatically plot hottest cantons
* show all data and filtered data (as in the gas/oil example)
* map of hottest area

#### Other
* newsletter for intersted people?

---

In [ ]:
#hide
from nbdev.showdoc import *
import pandas as pd
import glob
import matplotlib.pyplot as plt

In [ ]:
#dfs = []
#
#folder = '../openZH_covid-19/fallzahlen_kanton_total_csv'
#folder_v2 = '../openZH_covid-19/fallzahlen_kanton_total_csv_v2'
#prefix_fn = 'COVID19_Fallzahlen_Kanton'
#postfix_fn = 'total.csv'
#
#print(f'{folder_v2}/{prefix_fn}_*_{postfix_fn}.csv')
#
#for f in glob.glob(f'{folder_v2}/{prefix_fn}_*_{postfix_fn}'):
#    dfs.append(pd.read_csv(f))
#
#df = pd.concat(dfs)
#df['date'] = pd.to_datetime(df['date'], dayfirst=True)
#
#df.head(1000)

## Check data and columns for changes

In [ ]:
def remove(lst, rem):
    '''
    Returns a list containing all elements of lst not present in rem.
    '''
    return list([i for i in lst if i not in rem])

In [ ]:
# column names of CH file as a list
df_ch_v2 = pd.read_csv('../openZH_covid-19/COVID19_Fallzahlen_CH_total_v2.csv')
col_CH = list(df_ch_v2.columns.values)

# column names of ZH file as a list
df_ZH_v2 = pd.read_csv('../openZH_covid-19/fallzahlen_kanton_total_csv_v2/COVID19_Fallzahlen_Kanton_ZH_total.csv')
col_ZH = list(df_ZH_v2.columns.values)

# current/previously available collumns
col_prev = ['date', 'time', 'abbreviation_canton_and_fl', 'ncumul_tested', 'ncumul_conf', 'new_hosp',
           'current_hosp', 'current_icu', 'current_vent', 'ncumul_released', 'ncumul_deceased', 'source',
           'current_isolated', 'current_quarantined']

print(f'columns present in col_CH only: {remove(col_CH, col_ZH)}')
print(f'columns present in col_ZH only: {remove(col_ZH, col_CH)}')
print(f'new available columns: {remove(col_CH, col_prev)}')

columns present in col_CH only: []
columns present in col_ZH only: ['current_quarantined_riskareatravel', 'current_quarantined_total']
new available columns: []


## App Methods

### Load cantonal csv-files and merge to total for Switzerland

In [ ]:
def add_diff_col(df, col, new_col):
    df[new_col] = df[col].diff()
    return df

def get_CH_data_total():
    # where to find the data files
    folder_v2 = '/Users/eandreas/projects/dev/covid-19/openZH_covid-19/fallzahlen_kanton_total_csv_v2'
    prefix_c_fn = 'COVID19_Fallzahlen_Kanton'
    prefix_fn = 'COVID19_Fallzahlen'
    postfix_fn = 'total.csv'

    dfs = []

    for c in CANTONS.values():
    
        if (c == "FL"):
            file = f'{folder_v2}/{prefix_fn}_{c}_{postfix_fn}'
        else:
            file = f'{folder_v2}/{prefix_c_fn}_{c}_{postfix_fn}'
    
        dfc = pd.read_csv(file)
        dfc['date'] = pd.to_datetime(dfc['date'], dayfirst=True)
    
        # mark current rows as reported by cantons
        dfc['reported'] = True
    
        # add rows for missing (unreported) days
        idx = pd.date_range(dfc.date.min(), dfc.date.max())
        dfc = dfc.set_index('date')
        dfc = dfc.reindex(idx)
        dfc.index.name = 'date'
        dfc.reset_index(level=0, inplace=True)

        # flag added rows as 'reported = False'
        dfc.loc[dfc['reported'] != True, 'reported'] = False
    
        # fill unreported numbers (NaN) with number of last reported
        dfc.fillna(method='ffill', inplace=True)
    
        # fill missing values at the beginning with zero
        dfc.fillna(value=0, inplace=True)
    
        # add some columns with calculated values of interest
        add_diff_col(dfc, 'ncumul_tested', 'new_tested')
        add_diff_col(dfc, 'ncumul_conf', 'new_conf')
        add_diff_col(dfc, 'ncumul_deceased', 'new_deceased')
        add_diff_col(dfc, 'current_hosp', 'delta_hosp')
        add_diff_col(dfc, 'current_icu', 'delta_icu')
        add_diff_col(dfc, 'current_vent', 'delta_vent')
        add_diff_col(dfc, 'current_isolated', 'delta_isolated')
        add_diff_col(dfc, 'current_quarantined', 'delta_quarantined')
        add_diff_col(dfc, 'ncumul_released', 'new_released')

        # add 7-day simple mean averages
        dfc['new_conf_SMA7'] = round(dfc['new_conf'].rolling(window=7, center=True).mean(), 1)
        dfc['current_hosp_SMA7'] = round(dfc['current_hosp'].rolling(window=7, center=True).mean(), 1)
    
        # append the dataframe and go on with the next canton
        dfs.append(dfc)

    # sum up all the cantons dataframes
    df_ch = pd.concat(dfs, sort=False)
    df_ch = df_ch.groupby('date').sum(min_count=1)
    df_ch.index.name = 'date'
    df_ch.reset_index(level=0, inplace=True)
    # the SMA7 gets broken for incomplete data (missing cantons) when summing up, so let's recalculate
    df_ch['new_conf_SMA7'] = round(df_ch['new_conf'].rolling(window=7, center=True).mean(), 1)
    df_ch['current_hosp_SMA7'] = round(df_ch['current_hosp'].rolling(window=7, center=True).mean(), 1)

    return df_ch

In [ ]:
df_ch = get_CH_data_total()
df_ch['new_conf'].tail()

NameError: name 'CANTONS' is not defined

In [ ]:
df_ch.ncumul_conf.tail()

### Load BAG data for testing information

In [ ]:
def get_BAG_data():
    df_bag = pd.read_csv('/Users/eandreas/projects/dev/covid-19/bag_data/bag_data.csv')

    # split datetime column into a date and a time column
    time_list = pd.to_datetime(df_bag['date'], dayfirst=True).dt.time
    df_bag.insert(loc=1, column='time', value=time_list)                 
    df_bag['date'] = pd.to_datetime(df_bag['date'], dayfirst=True).dt.date
    df_bag['date'] = pd.to_datetime(df_bag['date'])

    # remove the rows without information (NaN for total_number_of_tests)
    df_bag = df_bag[df_bag['total_number_of_tests'].notnull()]
    
    # add column with number of positive and negative tests
    df_bag['ncumul_test_pos'] = round(df_bag['total_number_of_tests'] * df_bag['positivity_rate_percent'] / 100, 0)
    df_bag['ncumul_test_neg'] = df_bag['total_number_of_tests'] - df_bag['ncumul_test_pos']
    
    # calculate and add a column wit newly tested cases per day
    add_diff_col(df_bag, 'total_number_of_tests', 'new_tested')
    add_diff_col(df_bag, 'ncumul_test_pos', 'new_test_pos')
    add_diff_col(df_bag, 'ncumul_test_neg', 'new_test_neg')
    
    # add pos_rate / day
    df_bag['new_positivity_rate_percent'] = round(100 * df_bag['new_test_pos'] / df_bag['new_tested'], 1)

    return df_bag

In [ ]:
df_bag = get_BAG_data()
df_bag.tail(20)

,date,time,total_number_of_tests,positivity_rate_percent,isolated,quarantined,quarantined_travel,source_file,ncumul_test_pos,ncumul_test_neg,new_tested,new_test_pos,new_test_neg,new_positivity_rate_percent
158,2020-08-11,08:00:00,847325.0,5.2,1331.0,4212.0,13994.0,COVID-19_Epidemiologische_Lage_Schweiz.pdf,44061.0,803264.0,4816.0,251.0,4565.0,5.2
159,2020-08-12,08:00:00,854885.0,5.2,1331.0,4446.0,18244.0,COVID-19_Epidemiologische_Lage_Schweiz.pdf,44454.0,810431.0,7560.0,393.0,7167.0,5.2
160,2020-08-13,08:00:00,861555.0,5.2,1357.0,4550.0,18086.0,COVID-19_Epidemiologische_Lage_Schweiz.pdf,44801.0,816754.0,6670.0,347.0,6323.0,5.2
161,2020-08-14,08:00:00,868886.0,5.2,1535.0,4553.0,15928.0,COVID-19_Epidemiologische_Lage_Schweiz.pdf,45182.0,823704.0,7331.0,381.0,6950.0,5.2
162,2020-08-15,08:00:00,876065.0,5.2,1568.0,4632.0,16131.0,COVID-19_Epidemiologische_Lage_Schweiz.pdf,45555.0,830510.0,7179.0,373.0,6806.0,5.2
163,2020-08-16,08:00:00,881683.0,5.2,1568.0,4632.0,16127.0,COVID-19_Epidemiologische_Lage_Schweiz.pdf,45848.0,835835.0,5618.0,293.0,5325.0,5.2
164,2020-08-17,08:00:00,884472.0,5.2,1568.0,4632.0,16125.0,COVID-19_Epidemiologische_Lage_Schweiz.pdf,45993.0,838479.0,2789.0,145.0,2644.0,5.2
165,2020-08-18,08:00:00,890289.0,5.1,1570.0,4754.0,16260.0,COVID-19_Epidemiologische_Lage_Schweiz.pdf,45405.0,844884.0,5817.0,-588.0,6405.0,-10.1
166,2020-08-19,08:00:00,901074.0,5.1,1853.0,6532.0,21546.0,COVID-19_Epidemiologische_Lage_Schweiz.pdf,45955.0,855119.0,10785.0,550.0,10235.0,5.1
167,2020-08-20,08:00:00,910283.0,5.1,1877.0,6667.0,21421.0,COVID-19_Epidemiologische_Lage_Schweiz.pdf,46424.0,863859.0,9209.0,469.0,8740.0,5.1


In [ ]:
df_bag[df_bag['date'] == '2020-08-09']

,date,time,total_number_of_tests,positivity_rate_percent,isolated,quarantined,quarantined_travel,source_file,ncumul_test_pos,ncumul_test_neg,new_tested,new_test_pos,new_test_neg,new_positivity_rate_percent
156,2020-08-09,08:00:00,840409.0,5.2,1301.0,4071.0,13856.0,COVID-19_Epidemiologische_Lage_Schweiz.pdf,43701.0,796708.0,4685.0,243.0,4442.0,5.2


### Load BAG report data

In [ ]:
def get_BAG_report_data():
    df_bag_report = pd.read_excel('data/200325_Datengrundlage_Grafiken_COVID-19-Bericht.xlsx', skiprows=6)
    df_bag_report['Datum'] = pd.to_datetime(df_bag_report['Datum'])
    df_bag_report.rename(
        columns = {
            'Datum':'date',
            'Fallzahlen pro Tag':'new_conf',
            'Fallzahlen pro Tag, kumuliert':'ncumul_conf',
            'Hospitalisationen pro Tag':'new_hosp',
            'Hospitalisationen pro Tag, Kumuliert':'ncumul_hosp',
            'Todesfälle pro Tag':'new_deceased',
            'Todesfälle pro Tag, kumuliert':'ncumul_deceased',
        }, inplace = True) 
    return df_bag_report

In [ ]:
df_bag_report = get_BAG_report_data()
df_bag_report

,date,new_conf,ncumul_conf,new_hosp,ncumul_hosp,new_deceased,ncumul_deceased
0,2020-02-24,1,1,5,5,NaN,NaN
1,2020-02-25,1,2,4,9,NaN,NaN
2,2020-02-26,10,12,9,18,NaN,NaN
3,2020-02-27,10,22,4,22,NaN,NaN
4,2020-02-28,10,32,5,27,NaN,NaN
...,...,...,...,...,...,...,...
184,2020-08-26,366,41203,4,4368,0.0,1723.0
185,2020-08-27,352,41555,3,4371,2.0,1725.0
186,2020-08-28,356,41911,1,4372,0.0,1725.0
187,2020-08-29,103,42014,0,4372,0.0,1725.0


### Merge CH and BAG data

In [ ]:
def get_data():
    df_ch = get_CH_data_total()
    df_bag = get_BAG_data()
    df = df_ch.merge(right=df_bag, how='left', on='date')
    df.fillna(value=0, inplace=True)
    return df

In [ ]:
df = get_data()
df.tail()

,date,ncumul_tested,ncumul_conf,new_hosp,current_hosp,current_icu,current_vent,ncumul_released,ncumul_deceased,current_isolated,...,TotalPosTests1,ninst_ICU_intub,time,total_number_of_tests,positivity_rate_percent,isolated,quarantined,quarantined_travel,source_file,new_tested
199,2020-08-09,61887.0,36643.0,3.0,120.0,20.0,10.0,6420.0,1989.0,950.0,...,68.0,62.0,08:00:00,840409.0,5.2,1301.0,4071.0,13856.0,COVID-19_Epidemiologische_Lage_Schweiz.pdf,4685.0
200,2020-08-10,61887.0,36839.0,3.0,122.0,20.0,9.0,6435.0,1990.0,879.0,...,68.0,62.0,08:00:00,842509.0,5.2,1301.0,4071.0,13856.0,COVID-19_Epidemiologische_Lage_Schweiz.pdf,2100.0
201,2020-08-11,61887.0,37101.0,4.0,115.0,22.0,9.0,6457.0,1991.0,978.0,...,68.0,62.0,08:00:00,847325.0,5.2,1331.0,4212.0,13994.0,COVID-19_Epidemiologische_Lage_Schweiz.pdf,4816.0
202,2020-08-12,61887.0,37310.0,4.0,116.0,25.0,14.0,6475.0,1991.0,1022.0,...,68.0,62.0,08:00:00,854885.0,5.2,1331.0,4446.0,18244.0,COVID-19_Epidemiologische_Lage_Schweiz.pdf,7560.0
203,2020-08-13,61887.0,28530.0,3.0,100.0,21.0,13.0,5318.0,1429.0,948.0,...,68.0,62.0,08:00:00,861555.0,5.2,1357.0,4550.0,18086.0,COVID-19_Epidemiologische_Lage_Schweiz.pdf,6670.0


In [ ]:
df.new_tested[df['date'] == '2020-08-09'], df.new_conf[df['date'] == '2020-08-09'], 100 / df.new_tested[df['date'] == '2020-08-09'] * df.new_conf[df['date'] == '2020-08-09']

(199    4685.0
 Name: new_tested, dtype: float64,
 199    84.0
 Name: new_conf, dtype: float64,
 199    1.792956
 dtype: float64)

### Download an prepare BAG data for testing

In [ ]:
import requests

URLs = {
    'BAG_test_data': 'https://www.bag.admin.ch/dam/bag/de/dokumente/mt/k-und-i/aktuelle-ausbrueche-pandemien/2019-nCoV/covid-19-basisdaten-labortests.xlsx.download.xlsx/Dashboard_3_COVID19_labtests_positivity.xlsx'
}   

def download_BAG_test_data():
    r = requests.get(URLs['BAG_test_data'], allow_redirects=True)
    fname = URLs['BAG_test_data'].split('/')[-1]
    open(fname, 'wb').write(r.content)

In [ ]:
def get_BAG_test_data():
    df_bag_test = pd.read_excel('Dashboard_3_COVID19_labtests_positivity.xlsx')
    df_bag_test = df_bag_test.drop('Replikation_dt', axis=1)
    df_bag_test_pos = df_bag_test[df_bag_test['Outcome_tests'] == 'Positive']
    df_bag_test_pos = df_bag_test_pos.drop('Outcome_tests', axis=1)
    df_bag_test_neg = df_bag_test[df_bag_test['Outcome_tests'] == 'Negative']
    df_bag_test_neg = df_bag_test_neg.drop('Outcome_tests', axis=1)
    df_bag_test = df_bag_test_pos.merge(right=df_bag_test_neg, on='Datum')
    df_bag_test = df_bag_test.rename(columns={"Datum": "date", "Number_of_tests_x": "positive", "Number_of_tests_y": "negative"})
    df_bag_test['pos_rate'] = round(100 * df_bag_test['positive'] / (df_bag_test['positive'] + df_bag_test['negative']), 1)
    df_bag_test['SMA_7'] = round(df_bag_test['pos_rate'].rolling(window=7, center=True).mean(), 1)
    return df_bag_test

In [ ]:
download_BAG_test_data()
df_bag_test = get_BAG_test_data()
df_bag_test.tail()

,date,positive,negative,pos_rate,SMA_7
196,2020-08-09,129,1904,6.3,4.1
197,2020-08-10,258,5799,4.3,4.3
198,2020-08-11,299,7011,4.1,NaN
199,2020-08-12,257,6328,3.9,NaN
200,2020-08-13,236,5232,4.3,NaN


### Make sure all date independent of source are of the same length

In [ ]:
def stretch_data_frames(dfs):
    
    min_date = dfs[0].date.min()
    max_date = dfs[0].date.max()
    
    for df in dfs:
        if (df.date.min() < min_date):
            min_date = df.date.min()
        if (df.date.max() > max_date):
            max_date = df.date.max()
    
    # add rows for missing (unreported) days
    idx = pd.date_range(min_date, max_date)
    for i in range(len(dfs)):
        dfs[i] = dfs[i].set_index('date')
        dfs[i] = dfs[i].reindex(idx)
        dfs[i].index.name = 'date'
        dfs[i].reset_index(level=0, inplace=True)
    
    return tuple(dfs)

In [ ]:
df_ch, df_bag_test = stretch_data_frames([df_ch, df_bag_test])

In [ ]:
df_ch.date.max(), df_bag_test.date.max()

(Timestamp('2020-08-14 00:00:00'), Timestamp('2020-08-14 00:00:00'))

In [ ]:
df_bag_test.tail()

,date,positive,negative,pos_rate,SMA_7
199,2020-08-10,258.0,5799.0,4.3,4.3
200,2020-08-11,299.0,7011.0,4.1,NaN
201,2020-08-12,257.0,6328.0,3.9,NaN
202,2020-08-13,236.0,5232.0,4.3,NaN
203,2020-08-14,NaN,NaN,NaN,NaN


In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=df_bag_test['Datum'], y=df_bag_test['new_tests']))

fig.show()

KeyError: 'Datum'

In [ ]:
fig = go.Figure(data=[
    go.Bar(name='positive', x=df_bag_test.Datum, y=df_bag_test.positive),
    go.Bar(name='negative', x=df_bag_test.Datum, y=df_bag_test.negative)
])
fig.update_layout(barmode='stack')

fig.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Bar(x=[1, 2, 3, 4], y=[0, 2, 3, 5])) # fill down to xaxis
fig.add_trace(go.Scatter(x=[1, 2, 3, 4], y=[3, 5, 1, 7], fill='tonexty')) # fill to trace0 y
fig.update_layout(hovermode="x unified")

fig.show()

In [ ]:
df_bag_test['new_tests'].cumsum()

0           3
1           7
2          20
3          36
4          65
        ...  
196    843466
197    849523
198    856833
199    863418
200    868886
Name: new_tests, Length: 201, dtype: int64

In [ ]:
# To display the figure defined by this dict, use the low-level plotly.io.show function
import plotly.io as pio

pio.show(figure)


In [ ]:
import plotly.express as px

#fig = px.bar(df_bag_test[df_bag_test['Datum'] >= '2020-06-01'], x="Datum", y=["pos_rate"], barmode="group")
fig = px.bar(df_bag_test, x="Datum", y=["pos_rate"], barmode="group")
#fig.update_layout(title_text='Daily confirmed cases - Switzerland')
fig.update_layout(showlegend=False)
fig.show()

## Switzerland

In [ ]:
# all cantons including FL
CANTONS = {
    "01": "AG",
    "02": "AI",
    "03": "AR",
    "04": "BE",
    "05": "BL",
    "06": "BS",
    #"07": "FL",
    "08": "FR",
    "09": "GE",
    "10": "GL",
    "11": "GR",
    "12": "JU",
    "13": "LU",
    "14": "NE",
    "15": "NW",
    "16": "OW",
    "17": "SG",
    "18": "SH",
    "19": "SO",
    "20": "SZ",
    "21": "TG",
    "22": "TI",
    "23": "UR",
    "24": "VD",
    "26": "VS",
    "27": "ZG",
    "28": "ZH"
}

In [ ]:
def add_diff_col(df, col, new_col):
    df[new_col] = df[col].diff()
    return df

#### Data

In [ ]:
folder_v2 = '/Users/eandreas/projects/dev/covid-19/openZH_covid-19/fallzahlen_kanton_total_csv_v2'
prefix_c_fn = 'COVID19_Fallzahlen_Kanton'
prefix_fn = 'COVID19_Fallzahlen'
postfix_fn = 'total.csv'

dfs = []

for c in CANTONS.values():
    
    #c = 'ZH'
    
    if (c == "FL"):
        file = f'{folder_v2}/{prefix_fn}_{c}_{postfix_fn}'
    else:
        file = f'{folder_v2}/{prefix_c_fn}_{c}_{postfix_fn}'
    
    dfc = pd.read_csv(file)
    dfc['date'] = pd.to_datetime(dfc['date'], dayfirst=True)
    
    # mark current rows as reported by cantons
    dfc['reported'] = True
    
    # add rows for missing (unreported) days
    idx = pd.date_range(dfc.date.min(), dfc.date.max())
    dfc = dfc.set_index('date')
    dfc = dfc.reindex(idx)
    dfc.index.name = 'date'
    dfc.reset_index(level=0, inplace=True)

    # flag added rows as 'reported = False'
    dfc.loc[dfc['reported'] != True, 'reported'] = False
    
    # fill unreported numbers (NaN) with number of last reported
    dfc.fillna(method='ffill', inplace=True)
    
    # fill missing values at the beginning with zero
    dfc.fillna(value=0, inplace=True)
    
    # add some columns with calculated values of interest
    add_diff_col(dfc, 'ncumul_tested', 'new_tested')
    add_diff_col(dfc, 'ncumul_conf', 'new_conf')
    add_diff_col(dfc, 'ncumul_deceased', 'new_deceased')
    add_diff_col(dfc, 'current_hosp', 'delta_hosp')
    add_diff_col(dfc, 'current_icu', 'delta_icu')
    add_diff_col(dfc, 'current_vent', 'delta_vent')
    add_diff_col(dfc, 'current_isolated', 'delta_isolated')
    add_diff_col(dfc, 'current_quarantined', 'delta_quarantined')
    add_diff_col(dfc, 'ncumul_released', 'new_released')
    
    # append the dataframe and go on with the next canton
    dfs.append(dfc)
    
    print(f'Canton: {c}; {len(pd.unique(dfc.ncumul_tested))}')
    
    #break

# sum up all the cantons dataframes
df_ch = pd.concat(dfs, sort=False)
df_ch = df_ch.groupby('date').sum()
df_ch.index.name = 'date'
df_ch.reset_index(level=0, inplace=True)

Canton: AG; 1
Canton: AI; 1
Canton: AR; 1
Canton: BE; 1
Canton: BL; 1
Canton: BS; 2
Canton: FR; 1
Canton: GE; 134
Canton: GL; 1
Canton: GR; 1
Canton: JU; 1
Canton: LU; 1
Canton: NE; 1
Canton: NW; 1
Canton: OW; 1
Canton: SG; 1
Canton: SH; 1
Canton: SO; 1
Canton: SZ; 4
Canton: TG; 3
Canton: TI; 1
Canton: UR; 2
Canton: VD; 1
Canton: VS; 1
Canton: ZG; 1
Canton: ZH; 1


In [ ]:
df_ch.ncumul_tested.tail()

163    0.0
164    0.0
165    0.0
166    0.0
167    0.0
Name: ncumul_tested, dtype: float64

In [ ]:
idx

DatetimeIndex(['2020-02-28', '2020-02-29', '2020-03-01', '2020-03-02',
               '2020-03-03', '2020-03-04', '2020-03-05', '2020-03-06',
               '2020-03-07', '2020-03-08',
               ...
               '2020-08-03', '2020-08-04', '2020-08-05', '2020-08-06',
               '2020-08-07', '2020-08-08', '2020-08-09', '2020-08-10',
               '2020-08-11', '2020-08-12'],
              dtype='datetime64[ns]', name='date', length=167, freq='D')

In [ ]:
df_ch = pd.read_csv('/Users/eandreas/projects/dev/covid-19//openZH_covid-19/COVID19_Fallzahlen_CH_total_v2.csv')
df_ch = df_ch.groupby('date').sum()

add_diff_col(df_ch, 'ncumul_tested', 'new_tested')
add_diff_col(df_ch, 'ncumul_conf', 'new_conf')
add_diff_col(df_ch, 'ncumul_deceased', 'new_deceased')
add_diff_col(df_ch, 'current_hosp', 'delta_hosp')
add_diff_col(df_ch, 'current_icu', 'delta_icu')
add_diff_col(df_ch, 'current_vent', 'delta_vent')
add_diff_col(df_ch, 'current_isolated', 'delta_isolated')
add_diff_col(df_ch, 'current_quarantined', 'delta_quarantined')
add_diff_col(df_ch, 'ncumul_released', 'new_released')

df_ch_calc = df.groupby('date').sum()
df_ch_calc.reset_index(level=0, inplace=True)

df_ch.tail()

NameError: name 'df' is not defined

In [ ]:
df.tail()

NameError: name 'df' is not defined

In [ ]:
%ls ../openZH_covid-19/COVID19_Fallzahlen_CH_total_v2.csv

../openZH_covid-19/COVID19_Fallzahlen_CH_total_v2.csv


In [ ]:
df.columns.values

NameError: name 'df' is not defined

In [ ]:
df_c = pd.concat(
    [
        df['date'],
        df['time'],
        df['abbreviation_canton_and_fl'],
        df['ncumul_conf'],
        df['new_conf'],
        df['ncumul_deceased'],
        df['new_deceased'],
        df['ncumul_tested'],
        df['new_tested'],
        #df['new_conf'] / df['new_tested'],
        df['new_hosp'],
        df['current_hosp'],
        df['delta_hosp'],
        df['current_icu'],
        df['delta_icu'],
        df['current_vent'],
        df['delta_vent'],
        df['ncumul_released'],
        df['new_released'],
        df['current_isolated'],
        df['delta_isolated'],
        df['current_quarantined'],
        df['delta_quarantined']
    ],
    axis = 1,
    keys = [
        'date',
        'time',
        'canton',
        'ncumul_conf',
        'new_conf',
        'ncumul_deceased'
        'new_deceased',
        'ncumul_tested',
        'new_tested',
        #'test_rate_pos',
        'new_hosp',
        'current_hosp',
        'delta_hosp',
        'current_icu',
        'delta_icu',
        'current_vent',
        'delta_vent',
        'ncumul_released',
        'new_released',
        'current_isolated',
        'delta_isolated',
        'current_quarantined',
        'delta_quarantined'
    ])

df_c

NameError: name 'df' is not defined

## Work in Progress

In [ ]:
df_ch.plot(x='date', y='new_conf', kind = 'bar')

KeyError: 'date'

In [ ]:
df_ch_calc.plot(x='date', y='new_conf', kind = 'bar')

NameError: name 'df_ch_calc' is not defined

## Unused

In [ ]:
df_ch[df_ch['date'] >= '20200701'].head()

KeyError: 'date'

In [ ]:
mask = (df['date'] == '2020-06-22')

df.loc[mask]

NameError: name 'df' is not defined

In [ ]:
mask = (df['abbreviation_canton_and_fl'] == 'AG')

df['new_tested'].loc(mask) = 1

SyntaxError: cannot assign to function call (<ipython-input-35-dc3caab8729d>, line 3)

In [ ]:
cantons = df.abbreviation_canton_and_fl.unique()

for c in cantons:
    print(c)
    df['new_tested'] = df['ncumul_tested'].diff()
    
    break

NameError: name 'df' is not defined

In [ ]:
df['new_tested'] = df['ncumul_tested'].diff()
df['new_conf'] = df['ncumul_conf'].diff()

df

NameError: name 'df' is not defined

In [ ]:
import datetime as dt

df_ch_calc['date'].max().value - df_ch_calc['date'].min().value


NameError: name 'df_ch_calc' is not defined

In [ ]:
dt.datetime(df_ch_calc['date'].max().value)

NameError: name 'df_ch_calc' is not defined

In [ ]:
df_ch = pd.read_csv('/Users/eandreas/projects/dev/covid-19//openZH_covid-19/COVID19_Fallzahlen_CH_total_v2.csv')
df_ch['date'] = pd.to_datetime(df_ch['date'])
df_ch = df_ch.groupby('date').sum()

# Create a row for every day
all_days = pd.date_range(df_ch.index.min(), df_ch.index.max(), freq='D')
df_ch = df_ch.reindex(all_days)
# Fill missing values with previous day's number
df_ch.fillna(method='pad', inplace=True)
# Now there are only missing values at the start
# of the series, so set them to zero
df_ch.fillna(value=0, inplace=True)

df_ch.index.name = 'date'
df_ch.reset_index(level=0, inplace=True)
add_diff_col(df_ch, 'ncumul_tested', 'new_tested')
add_diff_col(df_ch, 'ncumul_conf', 'new_conf')
add_diff_col(df_ch, 'ncumul_deceased', 'new_deceased')
add_diff_col(df_ch, 'current_hosp', 'delta_hosp')
add_diff_col(df_ch, 'current_icu', 'delta_icu')
add_diff_col(df_ch, 'current_vent', 'delta_vent')
add_diff_col(df_ch, 'current_isolated', 'delta_isolated')
add_diff_col(df_ch, 'current_quarantined', 'delta_quarantined')
add_diff_col(df_ch, 'ncumul_released', 'new_released')

,date,ncumul_tested,ncumul_conf,new_hosp,current_hosp,current_icu,current_vent,ncumul_released,ncumul_deceased,current_isolated,current_quarantined,new_tested,new_conf,new_deceased,delta_hosp,delta_icu,delta_vent,delta_isolated,delta_quarantined,new_released
0,2020-01-24,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-25,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-01-26,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-01-27,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-01-28,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,2020-08-09,0.0,28985.0,0.0,56.0,6.0,7.0,3135.0,1389.0,731.0,1780.0,0.0,-1985.0,-163.0,-9.0,-4.0,0.0,-8.0,-44.0,5.0
199,2020-08-10,0.0,36919.0,0.0,106.0,16.0,7.0,5125.0,1907.0,879.0,3194.0,0.0,7934.0,518.0,50.0,10.0,0.0,148.0,1414.0,1990.0
200,2020-08-11,0.0,30939.0,1.0,77.0,11.0,6.0,5147.0,1483.0,912.0,2215.0,0.0,-5980.0,-424.0,-29.0,-5.0,-1.0,33.0,-979.0,22.0
201,2020-08-12,0.0,22895.0,0.0,94.0,20.0,12.0,5165.0,1134.0,557.0,965.0,0.0,-8044.0,-349.0,17.0,9.0,6.0,-355.0,-1250.0,18.0


In [ ]:
df_ch.tail()

,date,ncumul_tested,ncumul_conf,new_hosp,current_hosp,current_icu,current_vent,ncumul_released,ncumul_deceased,current_isolated,current_quarantined,new_tested,new_conf,new_deceased,delta_hosp,delta_icu,delta_vent,delta_isolated,delta_quarantined,new_released
198,2020-08-09,0.0,28985.0,0.0,56.0,6.0,7.0,3135.0,1389.0,731.0,1780.0,0.0,-1985.0,-163.0,-9.0,-4.0,0.0,-8.0,-44.0,5.0
199,2020-08-10,0.0,36919.0,0.0,106.0,16.0,7.0,5125.0,1907.0,879.0,3194.0,0.0,7934.0,518.0,50.0,10.0,0.0,148.0,1414.0,1990.0
200,2020-08-11,0.0,30939.0,1.0,77.0,11.0,6.0,5147.0,1483.0,912.0,2215.0,0.0,-5980.0,-424.0,-29.0,-5.0,-1.0,33.0,-979.0,22.0
201,2020-08-12,0.0,22895.0,0.0,94.0,20.0,12.0,5165.0,1134.0,557.0,965.0,0.0,-8044.0,-349.0,17.0,9.0,6.0,-355.0,-1250.0,18.0
202,2020-08-13,0.0,9192.0,0.0,24.0,10.0,4.0,2826.0,609.0,50.0,70.0,0.0,-13703.0,-525.0,-70.0,-10.0,-8.0,-507.0,-895.0,-2339.0


In [ ]:
df_ch = pd.read_csv('/Users/eandreas/projects/dev/covid-19//openZH_covid-19/COVID19_Fallzahlen_CH_total_v2.csv')
df_ch['date'] = pd.to_datetime(df_ch['date'])

# Create a row for every day
all_days = pd.date_range(df_ch.date.min(), df_ch.date.max(), freq='D')
df_ch = df_ch.reindex(all_days)
# Fill missing values with previous day's number
df_ch.fillna(method='pad', inplace=True)
# Now there are only missing values at the start
# of the series, so set them to zero
#df_ch.fillna(value=0, inplace=True)

df_ch.head()

,date,time,abbreviation_canton_and_fl,ncumul_tested,ncumul_conf,new_hosp,current_hosp,current_icu,current_vent,ncumul_released,ncumul_deceased,source,current_isolated,current_quarantined
2020-01-24,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-25,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-26,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-27,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-28,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
all_days

DatetimeIndex(['2020-01-24', '2020-01-25', '2020-01-26', '2020-01-27',
               '2020-01-28', '2020-01-29', '2020-01-30', '2020-01-31',
               '2020-02-01', '2020-02-02',
               ...
               '2020-08-04', '2020-08-05', '2020-08-06', '2020-08-07',
               '2020-08-08', '2020-08-09', '2020-08-10', '2020-08-11',
               '2020-08-12', '2020-08-13'],
              dtype='datetime64[ns]', length=203, freq='D')